[1. 데이터 설명 및 보기](#1) <br>
[2. 데이터 기초 분석/탐색](#2) <br>
[3. 데이터 클린징](#3) <br>
[4. 데이터 시각화](#4) <br>
[5. 통계 또는 머신러닝](#5) <br>
[6. 결론 및 본인 차별화 포인트](#6) <br>


***<div class="alert alert-block alert-info"> <a id =1> </a> 1. 데이터 설명 및 보기 </div>***
    !!!기존 데이터 사용 사례 포함/ 데이터 출처 미 인용 시 부정행위 판단

데이터 출처 => https://www.kaggle.com/tmdb/tmdb-movie-metadata?select=tmdb_5000_movies.csv <br>


In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np

file_path = "/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv"
df = pd.read_csv(file_path)
df.head(3)

영화 각각에 대한 정보들과 예산 및 수입, 그 영화와 어울리는 키워드 태그라인, 흥행도, 평점 및 평가자수가 담긴 데이터 csv 파일이다.<br>
해당 csv 데이터 파일은 https://www.themoviedb.org/ 사이트에서 대략 5000개 내외 추출하여 kaggle사이트의 Dataset에 공개된 업로드된 파일중 하나이다.<br>
이 영화 데이터들을 사용하여 https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system/input 와 같이 넷플릭스가 구현한 영화 추천 머신러닝 시스템을 직접 구축해보는 사례 등 여러 응용사례가 있다.<br>

(데이터 설명)<br>
budget- 해당 영화의 제작 예산이다. 아바타 영화의 예산은 $237 million 이다.<br>
genres- 그 영화가 해당하는 장르들이다. 배열 안에 그 영화가 해당하는 장르들이 Object로 담긴다. 아바타 영화의 장르는 다음과 같다.<br>
===> [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}] <br>
homepage- 해당 영화 홈페이지 사이트이다.<br>
id- 영화의 고유한 id이다. 캐글 사이트에서 제공하는 다른 csv파일에 있는 movie id와 일치한다.<br>
keywords- 그 영화의 주요 키워드들이다. 배열 안에 키워드들이 Object로 담긴다. 아바타 영화의 키워드는 다음과 같다.<br>
===> [{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, ...]<br>

그 외의 feature들은 수가 많아 간략하게 설명한다.<br>
original_language - 영화의 제작 원본언어 ////
original_title - 영화의 원본명 ////
overview - 대략적인 줄거리 ////
popularity - 흥행도 ////
production_companies - 제작사 ////
production_countries - 제작국가 ////
release_date - 개봉일 ////
revenue - 수입 ////
runtime - 러닝타임 ////
spoken_languages - 구어 언어 ////
status - 개봉을 했는가 ////
tagline - 영화를 제일 잘 표현하는 태그 문장 ////
title - 영화 제목 ////
vote_average - 10점 만점중 평점 ////
vote_count - 평점을 기록한 사람 수 

***<div class="alert alert-block alert-info"> <a id =2> </a>2.데이터 기초 분석/탐색 </div>***



In [ ]:
# 데이터를 먼저 백업하겠다.
movie_copy_backup = df.copy()
df.to_csv('./tmdb_5000_movies_backup.csv')

In [ ]:
df.info()

In [ ]:
df.index

다음의 데이터는 이번 분석에서 활용하지 않아 제거할 피쳐들이다.<br>
(1)genres (2)homepage, (3)id, (4)keywords, (5)original_language, (6)original_title, (7)overview, (9)production_companies, (10)production_countries, (14)spoken_languages, (15)status, (16)tagline <br>

다음의 데이터가 제거되지 않을 피쳐들이다.<br>
(0)budget-int64 (8)popularity-float64 (11)release_date-object (12)revenue-int64 (13)runtime-float64 (17)title-object (18)vote_average-float64 (19)vote_count-int64<br>
이 중, (11)release_date는 timestamp형식이 더 어울리므로 datetime 형식으로 바꾸겠다.<br>

In [ ]:
df.drop(['genres','homepage','id','keywords','original_language','original_title','overview','production_companies','production_countries','spoken_languages','status','tagline'], axis=1, inplace=True)
df['release_date'] = pd.to_datetime(df['release_date'])
df.dtypes

In [ ]:
# 타임스탬프에서 연도만 추출하여 새로운 변수로 만들겠다. 이 변수를 int64 데이터타입으로 한번 더 캐스팅할 것이다.
df['release_date_year'] = df['release_date'].dt.year

In [ ]:
# 다음과 같이 빈 내용의 결측데이터가 있어 없애겠다.
df[df['release_date_year'].isnull() == True]

In [ ]:
df.drop(4553, inplace=True)
df[df['release_date_year'].isnull() == True]

In [ ]:
# 타임스탬프 형식을 in64형식으로 캐스팅한다. 연도별 데이터 바이닝하기 위함이다.
df['release_date_year'] = df['release_date_year'].astype('int64')

In [ ]:
df.head(1)

In [ ]:
# 제작 예산이 많이 투입된 영화를 나열하여 수입 및 평점 등을 알아보자.
df.sort_values(by=['budget'], ascending=False, axis=0)[['release_date_year','title','budget','revenue','vote_average','vote_count','popularity']].head(3)

In [ ]:
# 다음과 같이 영화 평점 순으로 나열하려고 하는데, 투표수 1개로 전체가 평가되는 신빙성이 없는 상황이다.
df.sort_values(by=['vote_average'], ascending=False, axis=0)[['release_date_year','title','budget','revenue','vote_average','vote_count','popularity']].head(3)

In [ ]:
# 위 문제를 해결하고자 vote_count가 평균 이상인 것만 나열하겠다. 그것을 reduced_df란 새로운 데이터프레임으로 만들겠다
vote_mean = df['vote_count'].mean()
vote_mean

In [ ]:
# 자리가 좁아서 3개만 추출하겠다. reduced_df의 결과는 총 1271 rows가 나온다. 
# 이 reduced_df 의 갯수도 충분히 많으므로 이제부터 이 데이터 프레임 위주로 조작하겠다.
reduced_df = df[df['vote_count'] >= vote_mean].sort_values(by=['vote_average'], ascending=False, axis=0)
reduced_df.head(3)

In [ ]:
# reduced_df를 추출해보면 release_date_year는 1936~2016년 까지의 값을 갖고 있다. 여기서 대략 10년씩 10개의 간격으로 쪼개 그룹화하는 데이터 바이닝 작업을 하겠다. 
bins = [1935, 1946, 1956, 1966, 1976, 1986, 1996, 2006, 2016]
labels = ['1936~1946','1947~1956','1957~1966','1967~1976','1977~1986','1987~1996','1997~2006','2007~2016']
df_year_group = pd.cut(reduced_df['release_date_year'], bins, labels)
reduced_df["release_date_group"] = df_year_group
reduced_df.head(2)

In [ ]:
# reduced_df를 추출해보면 vote_average도 4.1~8.5의 값을 갖는다. 0.5 간격으로 쪼개는 데이터 바이닝을 하겠다.
bins = [4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5]
labels = ['4점초반','4점후반','5점초반','5점후반','6점초반','6점후반','7점초반', '7점후반','8점초반']
df_vote_group = pd.cut(reduced_df['vote_average'], bins, labels)
reduced_df['vote_average_group'] = df_vote_group
reduced_df.head(2)

In [ ]:
# 다음을 보면, 현대에 들어오면서 영화에 대한 사람들의 평가가 낮아지고 있다.
reduced_df.groupby(["release_date_group"])['vote_average'].mean()

***<div class="alert alert-block alert-info"> <a id =3> </a>3. 데이터 클린징</div>***



In [ ]:
# 결측치는 없다.
reduced_df.isnull().sum()

In [ ]:
reduced_df.dtypes

In [ ]:
# 왜 현대에 들어와서 vote_count에 이상치가 저렇게 많은가? 라는 고심 끝에, 인터넷의 발달로 영화가 흥행에 성공하여 world-wide해지면 이러한 이상치가 나오는 것은 당연한 현상이라 생각하여, 이상치를 처리하지 않겠다.
sns.boxplot(x="release_date_group", y="vote_count", data=reduced_df)

In [ ]:
# 중복치는 title컬럼을 제외하고는 있어도 크게 의미가 없으므로 고려하지 않는다.
reduced_df[reduced_df.duplicated('title') == True]

***<div class="alert alert-block alert-info"> <a id =4> </a>4. 데이터 시각화 </div>***



In [ ]:
reduced_df.head(1)

In [ ]:
# 평점(가로축)과 제작예산(세로축)의 관계를 그려보자.
sns.jointplot(x="vote_average", y="budget", data=reduced_df)

평점이 높은 영화의 예산이, 오히려 평점이 평균인 영화의 예산들보다 덜 투입되어가는 경향을 알 수 있다

In [ ]:
sns.jointplot(x="budget", y="revenue", data=reduced_df)

예산이 많이 투입될수록 수입이 증가하는 추세를 보인다

In [ ]:
pvt = reduced_df.pivot_table(index="release_date_group", columns="vote_average_group", aggfunc="size")

In [ ]:
sns.heatmap(pvt, cmap=sns.light_palette("gray", as_cmap=True), annot=True, fmt="d")

21세기에 가까워 질수록 투표수가 확실히 높아졌음과 동시에, 평점을 좋게 주는 확률이 점점 희박해짐을 알 수 있다

In [ ]:
reduced_df.plot(x='release_date', y='budget')

영화도 인터넷과 함께 성장을 한 산업이므로, 1990년대부터 급격한 예산투자가 시작되었다.

***<div class="alert alert-block alert-info"> 5. 통계 또는 머신러닝 <div>***



#### 상관분석

In [ ]:
reduced_df.head(1)

연속형 변수들 사이의 상관성을 파악하기 위해 상관분석을 하겠다.

In [ ]:
# 산점도로 스캐터플롯을 그려보자
sns.lmplot(x='budget', y='vote_average', data=reduced_df)

영화 예산과 평점의 상관계수가 약한 음의 상관관계임을 확인할 수 있다. 영화 예산을 더 투입할수록 영화의 평점이 낮아지는 경향이다

In [ ]:
#상관계수를 전체적으로 파악하자. 예상했듯 위의 그래프에 상응하는 상관계수값이 추출되었다. 
reduced_df.corr()

In [ ]:
sns.clustermap(reduced_df.corr() , annot=True, cmap='RdYlBu_r', vmin=-1, vmax=1)

그 외에도, revenue와 vote_average의 상관관계가 무상관 관계임은, 영화에 대한 평점은 수익과 상관없다는 것이라는 주목할 만한 결과이다.<br>release_date_year와 vote_average의 음의 상관관계에서도 시간이 지남에 따라 평가자들의 평점이 점점 낮아지는 경향을 재확인 해 주었다. <br>
revenue와 vote_count의 뚜렷한 양의 상관관계에서도 수익이 많은 영화가 많은 평가를 받았음을 알 수 있다.<br>
종합적으로 판단해 볼 때, 수익이 많은 영화에 많은 사람들이 평가를 하긴 하였는데 그 평가가 과거보다 평점을 낮게 하는 평가이고, 평점은 영화 수익과 상관이 없다는 점으로 보아, 관객들이 시간이 지남에 따라 까다로운 movie taste를 갖추게 되어 감을 추측할 수 있다.

#### 회귀분석

In [ ]:
# 시간이 지남에 따라 다음과 같이 영화들의 평점이 낮아진다.
sns.lmplot(x='release_date_year', y='vote_average', data=reduced_df)

최신으로 시간이 도달할 수록, 영화들의 평점이 낮아지는 이 상관성이 유의미한 인과관계를 갖는 것인지에 대하여<br>
release_date_year(개봉년도)를 독립변수로, vote_average(평점)을 종속변수로 정의하겠다<br>

In [ ]:
# 최소제곱법을 사용한 단순선형회귀 모형을 만들어 평가하겠다.
import statsmodels.api as sm
lin_reg = sm.OLS.from_formula("vote_average ~ release_date_year", reduced_df).fit()
lin_reg.summary()

1. coef를 통해 '영화평점 = -0.0242 * 개봉년도 + 55.2738'의 선형회귀식을 만들 수 있다.<br>
2. 독립변수 개봉년도의 유의확률 P>|t| 값이 0.05보다 작은 0이므로, 개봉년도는 영화평점에 유의미하게 영향을 미친다고 할 수 있다.<br>
3. R-squared의 값이 0.136이다. 위에서 만든 선형회귀식의 모델이 사실 별로 설명력이 없는 상태임을 짐작할 수 있다.<br>
4. Durbin-Watson의 값이 0.249이다. 0에 가까운 이 값은 잔차들이 양의 자기상관을 가진다는 의미이므로, 옳은 회귀모형이 아님을 의미한다. 이로 인해 t값이 실제보다 증가되어 위 2번의 왜곡된 결과를 유발했을 것이다.

***<div class="alert alert-block alert-info">6. 결론 및 본인 차별화 포인트 </div>***



그 외에도, revenue와 vote_average의 상관관계가 무상관 관계임은, 영화에 대한 평점은 수익과 상관없다는 것이라는 주목할 만한 결과이다.<br>release_date_year와 vote_average의 음의 상관관계에서도 시간이 지남에 따라 평가자들의 평점이 점점 낮아지는 경향을 재확인 해 주었다. <br>
revenue와 vote_count의 뚜렷한 양의 상관관계에서도 수익이 많은 영화가 많은 평가를 받았음을 알 수 있다.<br>
종합적으로 판단해 볼 때, 수익이 많은 영화에 많은 사람들이 평가를 하긴 하였는데 그 평가가 과거보다 평점을 낮게 하는 평가이고, 평점은 영화 수익과 상관이 없다는 점으로 보아, 관객들이 시간이 지남에 따라 까다로운 movie taste를 갖추게 되어 감을 추측할 수 있다.

### 결론

release_date_year(개봉년도)와 vote_average(평점)이 음의 상관관계를 보였고, 이에 따라 인과관계를 파악한 결과 이 둘은 사실상 인과관계가 없음이 밝혀지었으며, 시간이 직접적인 평점의 원인변수가 아니고, 시간이 흐름에 따라 생긴 다른 무엇인가의 요인들이 영향을 주었을 가능성이 높다고 결론을 짓는다.<br>
revenue(수익)와 vote_average(평점)은 무 상관 관계였으며, 이는 수익이 높은 영화들이 작품성이 좋을 것이란 편견을 깨주는 의미있는 결론이었다.

### 차별화 포인트

연도별로 영화에 대한 평가 투표수가 현재에 가까울 수록 이상치의 갯수가 급상승 하였으나, 인터넷의 발달 후 모바일이 일상인 요즈음 영화에 대한 평가가 손 쉬워졌다는 점에 따라, 이상치를 굳이 제거하지 않겠다는 판단을 하였다.<br>
영화 수입과 평점이 상관관계가 없다는 결과를 얻어, 사실상 영화관에서 흥행하는 영화가 작품성과는 상관없다는 통념을 깨는 결과를 도출해 내었다.